### Training the SNEE model

Setting up the environment

In [ ]:
# Clone the repository
!git clone https://github.com/Ggenoni/SENN.git

In [ ]:
# Install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local/miniconda
import os
os.environ["PATH"] += ":/usr/local/miniconda/bin"

In [ ]:
!conda --version

In [ ]:
# Change directory
%cd SENN

In [ ]:
# Create the virtual environment
!conda env create -f environment.yml

In [ ]:
!source /usr/local/miniconda/bin/activate senn && which python

In [ ]:
# ==> After running this cell in Colab, restart the runtime changing kernell <==

!source /usr/local/miniconda/bin/activate senn
!pip install ipykernel
!python -m ipykernel install --user --name=senn --display-name "Python (senn)"

In [ ]:
# Check for plt.style.use strings in the code 
import os

def search_in_files(directory, search_string):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".py"):  # Search only in Python files
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    for i, line in enumerate(f):
                        if search_string in line:
                            print(f"Found in {file_path}, line {i + 1}: {line.strip()}")

# Replace '.' with the directory you want to search (e.g., '/content/SENN')
search_in_files('/content/SENN', 'plt.style.use')


In [ ]:
# Add the style file
!mkdir -p ~/.config/matplotlib/stylelib/
!echo -e "axes.titlesize: 18\naxes.labelsize: 14\nfigure.dpi: 100" > ~/.config/matplotlib/stylelib/seaborn-paper.mplstyle


Training the model

In [ ]:
# Train the model with our config file

!source /usr/local/miniconda/bin/activate senn && python main.py --config configs/config.json